In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('/home/users/pjh/scripts/python_genome_packages')
import pprint

import pysam

from handygenome.common import ChromDict, Vcfspec

FASTA_PATH_HG19 = "/home/users/data/01_reference/human_g1k_v37/human_g1k_v37.fasta"
FASTA_PATH_HG38 = "/home/users/data/01_reference/human_g1k_v38/Homo_sapiens_assembly38.fasta"

FASTA_HG19 = pysam.FastaFile(FASTA_PATH_HG19)
FASTA_HG38 = pysam.FastaFile(FASTA_PATH_HG38)
CHROMDICT_HG19 = ChromDict(fasta_path=FASTA_PATH_HG19)
CHROMDICT_HG38 = ChromDict(fasta_path=FASTA_PATH_HG38)

In [2]:
from handygenome.readplus import readplus
import handygenome.igv
from handygenome.variantplus import breakends
from handygenome.readplus import readhandler
from handygenome.readplus import alleleinfosetup_sv

In [3]:
nbam_path = '/home/users/hspark/Projects/11_fusion_samples/tmp/LU-ALK49-N.s.md.ir.bam'
tbam_path = '/home/users/hspark/Projects/11_fusion_samples/tmp/LU-ALK49-T.s.md.ir.bam'

#nbam_path = '/home/users/hspark/Projects/11_fusion_samples/bam/wgs_smc2/LU-ALK49-N.s.md.ir.bam'
#tbam_path = '/home/users/hspark/Projects/11_fusion_samples/bam/wgs_smc2/LU-ALK49-T.s.md.ir.bam'

nbam = pysam.AlignmentFile(nbam_path)
tbam = pysam.AlignmentFile(tbam_path)

In [4]:
igv = handygenome.igv.IGV(60387)

In [5]:
igv.cmd('new')
igv.load([tbam_path, nbam_path])

OK
OK
OK


In [9]:
#vcfspec = Vcfspec('7', 55_176_696, 'G', ['C'])
vcfspec = Vcfspec('7', 55_182_730, 'G', ['A'])

In [7]:
bnds_dup = breakends.Breakends('1', 59_000_242, True, '1', 78_401_807, False, FASTA_HG19)
bnds_dup.set_pos_range0s()
#bnds_dup_new = breakends.Breakends('1', 59_000_240, True, '1', 78_401_806, False, FASTA_HG19)
bnds_inv = breakends.Breakends('1', 78_147_272, True, '1', 83_140_336, False, FASTA_HG19)
bnds_inv.set_pos_range0s()

In [10]:
#igv.goto([bnds_dup], width=50)
igv.goto([vcfspec])

OK


In [11]:
igv.cmd('group PAIR_ORIENTATION')
igv.cmd('group')

OK
OK


In [16]:
rpplist = readplus.get_rpplist_nonsv(tbam, FASTA_HG19, CHROMDICT_HG19, vcfspec.chrom, vcfspec.pos - 1,
                                     vcfspec.pos)

[KST 2022-05-06 17:08:16 INFO] ReadPlusPairList: Beginning initial fetch
[KST 2022-05-06 17:08:16 INFO] ReadPlusPairList: Beginning refined fetch
[KST 2022-05-06 17:08:16 INFO] ReadPlusPairList: Beginning assembly into readpluspair


In [23]:
rpplist.update_alleleinfo(vcfspec)

In [24]:
rpplist

[<ReadPlusPair object (qname: A01146:132:H277LDSX3:1:2340:31385:7279; rp1_pos: 5:26,220,778; rp2_pos: 7:55,182,646; alleleinfo: {<Vcfspec (7:55182730 G>A)>: {'alleleclass': 1}})>,
 <ReadPlusPair object (qname: A01146:132:H277LDSX3:1:1251:18855:36683; rp1_pos: 2:33,141,313; rp2_pos: 7:55,182,599; alleleinfo: {<Vcfspec (7:55182730 G>A)>: {'alleleclass': 1}})>,
 <ReadPlusPair object (qname: A01146:132:H277LDSX3:1:1251:18846:36699; rp1_pos: 2:33,141,320; rp2_pos: 7:55,182,599; alleleinfo: {<Vcfspec (7:55182730 G>A)>: {'alleleclass': 1}})>,
 <ReadPlusPair object (qname: A01146:132:H277LDSX3:1:2547:30698:34929; rp1_pos: 7:55,182,278; rp2_pos: 7:55,182,581; alleleinfo: {<Vcfspec (7:55182730 G>A)>: {'alleleclass': 1}})>,
 <ReadPlusPair object (qname: A01146:132:H277LDSX3:1:2165:7780:20509; rp1_pos: 7:55,182,302; rp2_pos: 7:55,182,594; alleleinfo: {<Vcfspec (7:55182730 G>A)>: {'alleleclass': 1}})>,
 <ReadPlusPair object (qname: A01146:132:H277LDSX3:1:2414:19334:16501; rp1_pos: 7:55,182,302; rp2

In [39]:
readlist = list(
    tbam.fetch(bnds_dup.chrom_bnd2, bnds_dup.pos_range0_bnd2.start - 10, bnds_dup.pos_range0_bnd2.stop + 10))

In [59]:
reads_filtered = [x for x in readlist if x.query_name.endswith(':23703')]
if len(reads_filtered) != 1:
    print('no')
read = reads_filtered[0]

In [60]:
read.query_name

'A01146:132:H277LDSX3:1:2266:31919:23703'

In [61]:
clippedat, traverses = alleleinfosetup_sv.check_read_spans_bnd(read, 
                                                               bnds_dup.chrom_bnd2, 
                                                               bnds_dup.pos_range0_bnd2, 
                                                               bnds_dup.endis5_bnd2)
print(clippedat, traverses)

False True


In [54]:
%%timeit

rpplist_bnd1 = readplus.ReadPlusPairList_old(bam=tbam, chrom=bnds_dup.chrom_bnd1, start0=bnds_dup.pos_range0_bnd1.start, 
                                             end0=bnds_dup.pos_range0_bnd1.stop, 
                                             fasta=FASTA_HG19, chromdict=CHROMDICT_HG19, mode='sv',
                                             endis5=bnds_dup.endis5_bnd1)
rpplist_bnd2 = readplus.ReadPlusPairList_old(bam=tbam, chrom=bnds_dup.chrom_bnd2, start0=bnds_dup.pos_range0_bnd2.start, 
                                             end0=bnds_dup.pos_range0_bnd2.stop, 
                                             fasta=FASTA_HG19, chromdict=CHROMDICT_HG19, mode='sv',
                                             endis5=bnds_dup.endis5_bnd2)

623 ms ± 6.29 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [55]:
%%timeit
rpplist_bnd1, rpplist_bnd2 = readplus.get_rpplist_sv(tbam, FASTA_HG19, CHROMDICT_HG19, bnds_dup)

253 ms ± 1.74 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
len(rpplist_bnd1)

643

In [26]:
len(rpplist_bnd2)

595

In [37]:
rpp = [rpp for rpp in rpplist_bnd2 if rpp.query_name.endswith(':30796')][0]

In [38]:
rpp

<ReadPlusPair object (qname: A01146:132:H277LDSX3:1:1448:20916:30796; rp1_pos: 1:78,401,482; rp2_pos: 1:78,401,718; alleleinfo: {})>

In [76]:
rpp = [rpp for rpp in rpplist_bnd2 if rpp.query_name.endswith(':15772:29309')][0]

In [77]:
rpp

<ReadPlusPair object (qname: A01146:132:H277LDSX3:1:1642:15772:29309; rp1_pos: 1:78,401,651; rp2_pos: 1:78,401,954; alleleinfo: {})>